# My Iggy

A simple `iggy`-like system.

In [31]:
%%clingo -n 0

% Description of the signed directed graph.
node(arcA; fnr; rpsP; rpmC).

% Possible signs of an edge.
sign(-1; 1).

% Given edges and their sign.
edge(arcA, fnr, -1).
edge(arcA, rpmC, -1).
edge(fnr, rpmC, 1).
edge(fnr, rpsP, -1).

% Deduce the input nodes.
input(J) :- node(J), not influence(J, _).

% Possible colors of a node.
color(-1; 1).

% Measurements of a node in 5 levels: -, dn, 0, up, +.
% Taking values (-2, -1, 0, 1, 2) to simplify description of RULE 1 below.
level(-2; -1; 0; 1; 2).

% Observed nodes and their color.
% observed(rpsP, 1).
observed(rpsP, -1).
observed(rpmC, -1).

% Define an alias for forbidden node coloring.
:- forbidden(N, C), node_color(N, C).

% Each node can only have 1 color.
1 {node_color(N, C): color(C)} :- node(N).
forbidden(N, C) :- node_color(N, D), color(C), color(D), D != C.

% Propagate effects forward.
% node_color(J, C) :- node(J), color(C), not forbidden(J, C).

% RULE 1: Observation satisfaction 
% For each observed node, the model behavior must match the biological observation data.
% That is:

% Where the model says should be 1, the data should >= 1;
forbidden(N, 1) :- observed(N, L), L < 1.

% Where the model says should be 0, the data should be -1 <= data <= 1;
forbidden(N, 0) :- observed(N, L), L < -1.
forbidden(N, 0) :- observed(N, L), L > 1.

% Where the model says should be -1, the data should <= -1.
forbidden(N, -1) :- observed(N, L), L > -1.

% RULE 2: Justification of the change in one predecessor
% For any non-input node, its change must be explained by the influence of
% at least one of its predecessors in the graph.
% More precisely, null changes can always occur independently of the state of
% the predecessor nodes. However, for observed changes in node activation levels,
% each must be explainable by an influence (with the appropriate sign) of
% at least one predecessor.

% For any non-input node J with color C, there must be
% at least one instance of influence(J, C).
forbidden(J, C) :- not influence(J, C), node_color(J, C), not input(J).

% We define the influence of node i on node j as:
% influence(i, j) := sign(i) * sign(edge(i, j))
influence(I, J, P*Q) :- edge(I, J, P), node_color(I, Q).
influence(J, C) :- influence(I, J, C).

% RULE 3: Justification of zero change
% A node can only exhibit zero change if it receives no or conflicting influences.
% This constraint therefore assumes that each influence has an effect,
% and that only contradictory influences can cancel each other out.
forbidden(N, 0) :- color(C), influence(N, C), not influence(N, -C).

% Repair model
% Each new influence (+ or -) must be consistent with itself,
% and at the same time inconsistent with the other.
new_influence(J, C) :- not not new_influence(J, C), not new_influence(J, -C),
    node(J), color(C).
influence(J, C) :- new_influence(J, C).

% Minimize the number of new influences (fixes) added to the graph.
#minimize{ 1, (J, C) : new_influence(J, C) }.
    
#show node_color/2.
% #show influence/2.
#show new_influence/2.

clingo version 5.4.0
Reading from stdin
Solving...
Answer: 1
node_color(fnr,-1) node_color(arcA,1) new_influence(arcA,1) new_influence(fnr,-1) new_influence(rpsP,-1) new_influence(rpmC,-1) node_color(rpsP,-1) node_color(rpmC,-1)
Optimization: 4
Answer: 2
node_color(fnr,-1) node_color(arcA,1) new_influence(arcA,1) new_influence(fnr,-1) new_influence(rpsP,-1) node_color(rpsP,-1) node_color(rpmC,-1)
Optimization: 3
Answer: 3
node_color(fnr,-1) node_color(arcA,1) new_influence(arcA,1) new_influence(rpsP,-1) node_color(rpsP,-1) node_color(rpmC,-1)
Optimization: 2
Answer: 4
node_color(fnr,-1) node_color(arcA,1) new_influence(rpsP,-1) node_color(rpsP,-1) node_color(rpmC,-1)
Optimization: 1
OPTIMUM FOUND

Models       : 4
  Optimum    : yes
Optimization : 1
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.001s
